In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import csv
import os
import re


In [2]:
headers = {
  'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'
}

url = "https://www.transfermarkt.pl/pko-ekstraklasa/startseite/wettbewerb/PL1"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
links = []

In [4]:
td_elements = soup.find_all('td', {'class': ['hauptlink no-border-links']})

In [5]:
for i in range(len(td_elements)):
    text = td_elements[i].find('a')['href']
    match = re.search(r'/(\d+)/', text)
    if match:
        number = match.group(1)
        links.append(number)

In [6]:
links

['9644',
 '238',
 '255',
 '324',
 '5689',
 '168',
 '428',
 '2300',
 '6112',
 '7146',
 '88',
 '759',
 '7154',
 '22431',
 '6110',
 '2118',
 '4000',
 '8936']

In [7]:
endpoint = "endpoint"

In [8]:
data = {
    'name': [],
    'appearances': [],
    'goals': [],
    'assists':[],
    'yellow': [],
    'yellow + red': [],
    'red': [],
    'in': [],
    'out': [],
    'pnm': [],
    'minutes': [],
}

In [9]:
for i in range(len(links)):
    club_url = "https://www.transfermarkt.pl/club_data/leistungsdaten/verein/" + endpoint + "/reldata/%262022/plus/1"
    new_club_url = club_url.replace(endpoint, links[i])    
    response = requests.get(new_club_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    endpoint = links[i]
    all_tr = soup.find_all('tr', {'class': ['odd', 'even']})
    for j in range(len(all_tr)):
        all_td = all_tr[j].find_all('td', {'class': ['zentriert']})
        data['name'].append(soup.find_all('span', {'class': ['show-for-small']})[j].find('a')['title'])
        data['appearances'].append( all_td[4].text )
        data['goals'].append( all_td[5].text )
        data['assists'].append( all_td[6].text )
        data['yellow'].append( all_td[7].text )
        data['yellow + red'].append( all_td[8].text )
        data['red'].append( all_td[9].text )
        data['in'].append( all_td[10].text )
        data['out'].append( all_td[11].text )
        data['pnm'].append( all_td[12].text )
        data['minutes'].append( all_tr[j].find_all('td', {'class': ['rechts']})[0].text )
    
    

In [10]:
data

{'name': ['Vladan Kovacevic',
  'Kacper Trelowski',
  'Xavier Dziekonski',
  'Jakub Rajczykowski',
  'Stratos Svarnas',
  'Zoran Arsenic',
  'Bogdan Racovitan',
  'Adrian Gryszkiewicz',
  'Tomas Petrasek',
  'Milan Rundic',
  'Andrzej Niewulis',
  'Oskar Krzyzak',
  'Fran Tudor',
  'Ivi López',
  'Vladyslav Kochergin',
  'Giannis Papanikolaou',
  'Deian Sorescu',
  'Ben Lederman',
  'Patryk Kun',
  'Bartosz Nowak',
  'Jean Carlos Silva',
  'Mateusz Wdowiak',
  'Gustav Berggren',
  'Wiktor Dlugosz',
  'Marcin Cebula',
  'Szymon Czyz',
  'Valeriane Gvilia',
  'Fábio Sturgeon',
  'Igor Sapala',
  'Tobiasz Kubik',
  'Olivier Sukiennicki',
  'Vladislavs Gutkovskis',
  'Fabian Piasecki',
  'Sebastian Musiolik',
  'Daniel Szelagowski',
  'Filip Bednarek',
  'Dominik Holec',
  'Krzysztof Bakowski',
  'Artur Rudko',
  'Karol Drazdzewski',
  'Mateusz Medrala',
  'Joel Pereira',
  'Filip Dagerstal',
  'Antonio Milic',
  'Lubomir Satka',
  'Pedro Rebocho',
  'Bartosz Salamon',
  'Mateusz Zukowski'

In [11]:
df = pd.DataFrame(data)
df

,name,appearances,goals,assists,yellow,yellow + red,red,in,out,pnm,minutes
0,Vladan Kovacevic,32,-,1,2,-,-,-,-,"2,28",2.910'
1,Kacper Trelowski,14,-,-,1,-,-,-,-,"2,29",1.290'
2,Xavier Dziekonski,Bez występów w tym sezonie,-,-,-,-,-,-,-,0,-
3,Jakub Rajczykowski,Bez występów w tym sezonie,-,-,-,-,-,-,-,0,-
4,Stratos Svarnas,39,3,4,6,-,-,-,2,"2,21",3.552'
...,...,...,...,...,...,...,...,...,...,...,...
670,Ángelo Henríquez,29,7,1,8,1,-,3,18,"0,79",2.156'
671,Kamil Zapolnik,12,-,-,1,-,-,9,3,"0,67",399'
672,Emmanuel Agbor,1,-,-,-,-,-,1,-,"1,00",44'
673,Wiktor Bogacz,Bez występów w tym sezonie,-,-,-,-,-,-,-,0,-


In [12]:
data_keys = data.keys()
new_2d_arr = np.array([data[i] for i in data_keys])

In [13]:
trans = np.transpose(new_2d_arr)

In [14]:
df = pd.DataFrame(trans)

In [15]:
df

,0,1,2,3,4,5,6,7,8,9,10
0,Vladan Kovacevic,32,-,1,2,-,-,-,-,"2,28",2.910'
1,Kacper Trelowski,14,-,-,1,-,-,-,-,"2,29",1.290'
2,Xavier Dziekonski,Bez występów w tym sezonie,-,-,-,-,-,-,-,0,-
3,Jakub Rajczykowski,Bez występów w tym sezonie,-,-,-,-,-,-,-,0,-
4,Stratos Svarnas,39,3,4,6,-,-,-,2,"2,21",3.552'
...,...,...,...,...,...,...,...,...,...,...,...
670,Ángelo Henríquez,29,7,1,8,1,-,3,18,"0,79",2.156'
671,Kamil Zapolnik,12,-,-,1,-,-,9,3,"0,67",399'
672,Emmanuel Agbor,1,-,-,-,-,-,1,-,"1,00",44'
673,Wiktor Bogacz,Bez występów w tym sezonie,-,-,-,-,-,-,-,0,-


In [16]:
cleansing = np.where(trans[:,:] == "-", 0, trans[:,:])

In [17]:
df = pd.DataFrame(cleansing)

In [18]:
df

,0,1,2,3,4,5,6,7,8,9,10
0,Vladan Kovacevic,32,0,1,2,0,0,0,0,"2,28",2.910'
1,Kacper Trelowski,14,0,0,1,0,0,0,0,"2,29",1.290'
2,Xavier Dziekonski,Bez występów w tym sezonie,0,0,0,0,0,0,0,0,0
3,Jakub Rajczykowski,Bez występów w tym sezonie,0,0,0,0,0,0,0,0,0
4,Stratos Svarnas,39,3,4,6,0,0,0,2,"2,21",3.552'
...,...,...,...,...,...,...,...,...,...,...,...
670,Ángelo Henríquez,29,7,1,8,1,0,3,18,"0,79",2.156'
671,Kamil Zapolnik,12,0,0,1,0,0,9,3,"0,67",399'
672,Emmanuel Agbor,1,0,0,0,0,0,1,0,"1,00",44'
673,Wiktor Bogacz,Bez występów w tym sezonie,0,0,0,0,0,0,0,0,0


In [19]:
header_columns = ["Imie i Nazwisko", "Występy", "Gole", "Asysty", "Żółte kartki", "Żółte + czerwone", "Czerwone kartki", "Wejścia", "Zejścia", "PNM", "Minuty"]

In [20]:
directory_file_csv = os.path.join(os.getcwd(), "League_Stats_Data.csv")

if os.path.isfile(directory_file_csv):
    df.to_csv('League_Stats_Data.csv', mode='a', index=False, header=False)
else:
    df.columns = header_columns
    df.to_csv("League_Stats_Data.csv", index = False)

In [21]:
data = pd.read_csv("League_Stats_Data.csv")

In [22]:
data

,Imie i Nazwisko,Występy,Gole,Asysty,Żółte kartki,Żółte + czerwone,Czerwone kartki,Wejścia,Zejścia,PNM,Minuty
0,Manuel Neuer,16,0,0,1,0,0,0,0,"2,31",1.440'
1,Yann Sommer,25,0,0,1,0,0,0,0,"1,88",2.250'
2,Sven Ulreich,8,0,0,2,0,0,0,0,"3,00",720'
3,Johannes Schenk,Bez występów w tym sezonie,0,0,0,0,0,0,0,0,0
4,Matthijs de Ligt,43,3,1,5,0,0,5,6,"2,09",3.390'
...,...,...,...,...,...,...,...,...,...,...,...
5094,Ángelo Henríquez,29,7,1,8,1,0,3,18,"0,79",2.156'
5095,Kamil Zapolnik,12,0,0,1,0,0,9,3,"0,67",399'
5096,Emmanuel Agbor,1,0,0,0,0,0,1,0,"1,00",44'
5097,Wiktor Bogacz,Bez występów w tym sezonie,0,0,0,0,0,0,0,0,0
